# The Predictions
I prepared a special method to predict the element abundances here, primarily due to the poorly optimized keras package.  
The problem is that (especially when you are using keras-gpu in jupyter notebook) loading a model into memory usually takes some time, but only a few computer resource is used. So, I write another script to store all the codes necessarily for the deep learning prediction, and load the trained model, and run many python programs simultaneously.  
Also, the GPU acceeration here is not useful, because the time is limited by I/O rather than calculation and GPUs are not always have more memory than RAM. 
So I specify CPU for this task. 
Be careful, it may cause a drastic increase in CPU load and RAM memory for a few seconds.  

Firstly, you need to check the existence of spectra. Also, the redshift and the extinctions shall be specified.  
Here I list the 11 HST spectra used in my paper.  
Moreover, you can choose some SN names here, and you can find the TARDIS readable element abundances with the as-chosen names in a directory (which will be specified later).  

In [1]:
import os
import glob
import yaml
import keras
import astropy
import numpy as np
import pandas as pd
import keras.backend as K
import astropy.units as u
import astropy.constants as c
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from keras.models import Sequential,Model
from dust_extinction.averages import GCC09_MWAvg
from sklearn.model_selection import train_test_split
from dust_extinction.parameter_averages import CCM89,F99

Using TensorFlow backend.


In [8]:
ProberRes=pd.read_csv('MdSaver/SmallRun/DataRes.csv')
RawElems=[str(i)+'_'+str(j) for i in range(6,29) for j in [1,2,3,4]]
ChosenElem=[]
for k in RawElems:
    if ProberRes[k][0]>0.1:continue
    ChosenElem.append(k)

In [2]:
spectralist=[
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_0.4d.dat',
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_-2.6d.dat',
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_3.7d.dat',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_-3.1d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_-1.1d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_0.9d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_3.9d.flm',
    'ObserveSpectra/Prediction/SN2011iv/SN2011iv_0.6d.flm',
    'ObserveSpectra/Prediction/SN2015F/SN2015F_-2.3d.flm',
    'ObserveSpectra/Prediction/ASASSN-14lp/ASASSN-14lp_-4.4d.flm',
    'ObserveSpectra/Prediction/SN2011by/SN2011by_-0.4d.flm']
ext1list=[CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),
         CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),
         CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),F99(Rv=3.1)]
ext2list=[GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),
         GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),
         GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg()]
ext1EbvList=[0,0,0,0.206,0.206,0.206,0.206,0,0.035,0.33,0.039]
ext2EbvList=[0,0,0,0.135,0.135,0.135,0.135,0,0.175,0.021,0.013]
zlist=[0.000804,0.000804,0.000804,0.00389,0.00389,0.00389,0.00389,0.006494,0.0049,0.0051,0.002843]
snname=['SN2011fe_0.4d','SN2011fe_-2.6d','SN2011fe_3.7d','SN2013dy_-3.1d','SN2013dy_-1.1d',
        'SN2013dy_0.9d','SN2013dy_3.9d','SN2011iv_0.6d','SN2015F_-2.3d','ASASSN-14lp_-4.4d','SN2011by_-0.4d']

## The environment
You may need to set the python environment, make sure it won't detect the GPU card, then run on CPU. 

In [ ]:
os.environ['LD_LIBRARY_PATH']='/home/hulei/anaconda3/pkgs/cudatoolkit-9.2-0/lib'
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']=''

## Keep On
The next step is a little bit stupid, you need to copy the lists in the above cell and write them into a string form manually.  
Then, run the cell and insert the observed spectra informations into the python script.  

In [5]:
ObservedSpectralInformation='''
spectralist=[
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_0.4d.dat',
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_-2.6d.dat',
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_3.7d.dat',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_-3.1d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_-1.1d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_0.9d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_3.9d.flm',
    'ObserveSpectra/Prediction/SN2011iv/SN2011iv_0.6d.flm',
    'ObserveSpectra/Prediction/SN2015F/SN2015F_-2.3d.flm',
    'ObserveSpectra/Prediction/ASASSN-14lp/ASASSN-14lp_-4.4d.flm',
    'ObserveSpectra/Prediction/SN2011by/SN2011by_-0.4d.flm']
ext1list=[CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),
         CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),
         CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),F99(Rv=3.1)]
ext2list=[GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),
         GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),
         GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg()]
ext1EbvList=[0,0,0,0.206,0.206,0.206,0.206,0,0.035,0.33,0.039]
ext2EbvList=[0,0,0,0.135,0.135,0.135,0.135,0,0.175,0.021,0.013]
zlist=[0.000804,0.000804,0.000804,0.00389,0.00389,0.00389,0.00389,0.006494,0.0049,0.0051,0.002843]
snname=['SN2011fe_0.4d','SN2011fe_-2.6d','SN2011fe_3.7d','SN2013dy_-3.1d','SN2013dy_-1.1d',
        'SN2013dy_0.9d','SN2013dy_3.9d','SN2011iv_0.6d','SN2015F_-2.3d','ASASSN-14lp_-4.4d','SN2011by_-0.4d']
'''
EnvironmentSpecifier='''
os.environ['LD_LIBRARY_PATH']='/home/hulei/anaconda3/pkgs/cudatoolkit-9.2-0/lib'
os.environ['CUDA_DEVICE_ORDER']='PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES']=''
'''
with open('PredictorOneElem.py','r') as scripter:
    ScriptProgram=scripter.read()
ScriptProgram=ScriptProgram.replace('######HereItIsTheSpecifier######',ObservedSpectralInformation)
ScriptProgram=ScriptProgram.replace('######TheEnvironmentSpecifier######',ObservedSpectralInformation)
with open('PredictorOneElemReallyRunner.py','w') as writter:
    writter.write(ScriptProgram)


## Let the Prediction Begin
As I am using "nohup" here, don't expect the results are available soon after the notebook finished its calculation.  
Here, you can specify the directory where the trained models are stored in "modelposition",  
and specify the directory where the testing set spectra is stored in "Xtextposition",  
and specify the directory where the testing set element abundance is stored in "Ytestposition",  
and specify the output element abundance at "observeoutposition".  
As before, you probably need to specify the conda environment, which is "GeronimoEstellarChen" here --apparently, you don't have my environment.  

On my workstation with 384 GB RAM and 48 cores, it takes about a minute that you can do nothing but staring the htop which shows full-load.  
If your computer is uncomfortable, you can set a timer here.  import time;time.sleep(10). 

In [ ]:
for k in ChosenElem:
    modelposition='MdSaver/HunKRun/'+k+'.hdf'
    Xtestposition='DataCache/2019.3.3/Xtest.npy'
    Youtposition='DataCache/2019.3.3/Yout'+k+'.npy'
    observeoutposition='DataCache/2019.3.3/ObsYout/'+k+'.npy'
    os.popen('source activate GeronimoEstellarChen \n'+
             'nohup python PredictorOneElemReallyRunner.py '+modelposition+' '+Xtestposition+' '+Youtposition+' '+observeoutposition+' > /dev/null &')

## The Results
Firstly, we shall read the results from the output files of the last cell, the "datareader" here matches the "observeoutposition" before.  

In [12]:
PredData=pd.DataFrame(np.zeros([11,35]),columns=['NameList']+ChosenElem)
PredData['NameList']=snname
for k in ChosenElem:
    datareader=np.load('DataCache/2019.3.3/ObsYout/'+k+'.npy')
    datareader=datareader.reshape(11)
    PredData[k]=datareader
Y_test=pd.read_csv('DataCache/2019.3.3/Ytest.csv',index_col=0)

## The Reverse and One-Sigma Limit
In the Reverser function, the output of neural network (ranging from 0 to 1) were remapped to the original multiplicaiton factor (ranging from 0 to 3).  
In the One-Sigma function, the 15th and 85th percentile of the testing dataset are calculated, we assume for a certain predicted value, its possible relating real value observes a normal distribution, then we calculate the one-sigma error like this. 

In [17]:
Yaverage=np.array([1.49088116, 1.49097508, 1.50731088, 1.50455924, 1.51035671,
       1.50782288, 1.51765324, 1.49176123, 1.49786083, 1.49164187,
       1.51008039, 1.50649329, 1.49706889, 1.48256907, 1.50291872,
       1.49671495, 1.48266993, 1.49328473, 1.49776985, 1.48788236,
       1.50800197, 1.49640525, 1.49671263, 1.48329884, 1.50654272,
       1.51546595, 1.50350355, 1.51359888, 1.50366677, 1.51347009,
       1.49226848, 1.49165777, 1.49447523, 1.50564056, 1.50430572,
       1.4965303 , 1.48774539, 1.50293614, 1.50234283, 1.49455285,
       1.49912642, 1.49035065, 1.49374915, 1.48790414, 1.49469701,
       1.51057406, 1.49123219, 1.49597135, 1.50784911, 1.49772004,
       1.5341486 , 1.5098752 , 1.5031406 , 1.48253015, 1.48185943,
       1.5142945 , 1.51335541, 1.49587574, 1.5064204 , 1.4859998 ,
       1.49735422, 1.49443917, 1.49713147, 1.50904882, 1.50430961,
       1.48899186, 1.4843948 , 1.50350289, 1.51124936, 1.50585086,
       1.51427044, 1.49639282, 1.50156886, 1.50700966, 1.49711655,
       1.50462769, 1.48889458, 1.50254653, 1.48807825, 1.49757542,
       1.20827367, 1.48634746, 1.49944391, 1.51260678, 1.19763427,
       1.51552949, 1.50722289, 1.49200582, 1.20885273, 1.48434282,
       1.49319914, 1.49298437])
Ystd=np.array([0.85998959, 0.86814719, 0.8687621 , 0.87050864, 0.86075304,
       0.86440532, 0.86272026, 0.86967863, 0.8626639 , 0.86734201,
       0.86057179, 0.87010535, 0.87411794, 0.8643628 , 0.86209518,
       0.86797349, 0.86725306, 0.85800528, 0.86381744, 0.86611775,
       0.8669094 , 0.86762243, 0.8675576 , 0.86361385, 0.86510062,
       0.8652972 , 0.86980369, 0.86254431, 0.87056344, 0.86437873,
       0.86389094, 0.87339658, 0.86688045, 0.86508027, 0.87134972,
       0.86946127, 0.86630519, 0.8612666 , 0.86535052, 0.86876499,
       0.85862661, 0.8706371 , 0.86687728, 0.86952742, 0.8607824 ,
       0.86498409, 0.85954263, 0.85826469, 0.86454108, 0.86876391,
       0.86799523, 0.86188565, 0.86329134, 0.86601738, 0.87293888,
       0.86968641, 0.86252809, 0.87444087, 0.86285946, 0.8640128 ,
       0.86677515, 0.87035309, 0.86068416, 0.86235551, 0.86720743,
       0.86189559, 0.86178644, 0.86734348, 0.86053826, 0.86848727,
       0.87080408, 0.86002404, 0.86218669, 0.85973503, 0.85875187,
       0.87255914, 0.86074365, 0.86338974, 0.85837912, 0.86682464,
       0.73932632, 0.85765432, 0.86773346, 0.87239269, 0.73557205,
       0.86274201, 0.87080225, 0.86799464, 0.73390453, 0.86346123,
       0.87181945, 0.87096847])

In [24]:
def Reverser(Yout,elem='26_4'):
    order=int(elem.split('_')[0])*4+int(elem.split('_')[1])-25
    Yraw=np.arctanh(Yout*2-1)*Ystd[order]+Yaverage[order]
    return Yraw
def OneSigmaCalculator(Yout,Yreal,center=0.5,window=0.05):
    Ylist=pd.DataFrame(np.array([Yout[:,0],np.array(Yreal)]).T,columns=['Yout','Yreal'])
    Yselect=Ylist[(Ylist['Yout']>center-window) & (Ylist['Yout']<center+window)].copy()
    if Yselect.shape[0]==0:
        OneMinusSigma=0
        MidData=0.5
        OnePlusSigma=1
    else:
        OneMinusSigma=np.percentile(Yselect['Yreal'],15.865,axis=0)
        MidData=np.percentile(Yselect['Yreal'],50,axis=0)
        OnePlusSigma=np.percentile(Yselect['Yreal'],84.135,axis=0)
    return OneMinusSigma,MidData,OnePlusSigma

## Calculate the Lower Limit and Upper Limit
Here it is the one-sigma lower limit and the upper limit of all elements. 

In [26]:
PredUpper=pd.DataFrame(np.zeros([11,35]),columns=['NameList']+ChosenElem)
PredUpper['NameList']=snname
PredMid=pd.DataFrame(np.zeros([11,35]),columns=['NameList']+ChosenElem)
PredMid['NameList']=snname
PredLower=pd.DataFrame(np.zeros([11,35]),columns=['NameList']+ChosenElem)
PredLower['NameList']=snname
for j in snname:
    for k in ChosenElem:
        lowerlimit,mid,upperlimit=OneSigmaCalculator(Yout=np.load('DataCache/2019.3.3/Yout'+k+'.npy'),Yreal=Y_test[k],center=np.array(PredData.loc[PredData['NameList']==j,k])[0],window=0.02)
        PredUpper.loc[PredUpper['NameList']==j,k]=upperlimit
        PredMid.loc[PredMid['NameList']==j,k]=mid
        PredLower.loc[PredLower['NameList']==j,k]=lowerlimit
RefineData,RefineUpper,RefineMid,RefineLower=PredData.copy(),PredUpper.copy(),PredMid.copy(),PredLower.copy()
for j in snname:
    for k in ChosenElem:
        RefineData.loc[RefineData['NameList']==j,k]=Reverser(Yout=np.array(PredData.loc[PredData['NameList']==j,k])[0],elem=k)
        RefineUpper.loc[RefineUpper['NameList']==j,k]=Reverser(Yout=np.array(PredUpper.loc[PredUpper['NameList']==j,k])[0],elem=k)
        RefineMid.loc[RefineMid['NameList']==j,k]=Reverser(Yout=np.array(PredUpper.loc[PredUpper['NameList']==j,k])[0],elem=k)
        RefineLower.loc[RefineLower['NameList']==j,k]=Reverser(Yout=np.array(PredLower.loc[PredUpper['NameList']==j,k])[0],elem=k)
RefineData.to_csv('DataCache/2019.3.3/RefineData.csv')
RefineUpper.to_csv('DataCache/2019.3.3/RefineUpper.csv')
RefineMid.to_csv('DataCache/2019.3.3/RefineMid.csv')
RefineLower.to_csv('DataCache/2019.3.3/RefineLower.csv')

## Element Solar Mass
Because I am using a IG model as the base and discussing the multiplicaiton factors during deep learning, now I can simply convert the multiplicaiton factor to the real mass in the unit of mass of sun.  
I firstly calculate the mass of each element in each zone in the IG model, then multiply the "multiplicaiton factor" onto it.  

In [31]:
solar_mass=1.98847*10**33#gram
dens=np.genfromtxt('IGmodel/Density.dat',skip_header=1)
mdd116=np.genfromtxt('IGmodel/Element.dat')

In [32]:
def ElemCal(Abundance,Density,VpH=7100,date=11.57,elem='26_3'):#to some reason the date seems useless. 
    ElemDens=Abundance[:,int(elem.split('_')[0])]*Density[:,2]
    zone=int(elem.split('_')[1])
    velogrid=0
    if zone==1:velogrid=np.linspace(VpH,10000,num=200)
    if zone==2:velogrid=np.linspace(10000,13200,num=200)
    if zone==3:velogrid=np.linspace(13200,17000,num=200)
    if zone==4:velogrid=np.linspace(17000,Density[:,1].max()-0.001,num=200)
    ElemDensFunc=interp1d(Density[:,1],ElemDens)#This Element Density is the density at 11.57 days. According to the homogenity when ejecta is expanding, we add a time**-3 part below. 
    radius=date*24*3600*velogrid*100000 #In centimeter
    ShellThickness=(velogrid[1]-velogrid[0])*date*24*3600*100000 # In centimeter
    Mass=np.sum(ElemDensFunc(velogrid)*(11.57/date)**3*4*np.pi*(radius)**2*ShellThickness)
    return Mass/solar_mass #in the unit of solar mass

## Calculate And Store
I store the mass of different elements in different zones into the "DataProduct" directory.  

In [35]:
MassData=pd.DataFrame(np.zeros([11,35]),columns=['NameList']+ChosenElem)
MassData['NameList']=snname
MassUpper=pd.DataFrame(np.zeros([11,35]),columns=['NameList']+ChosenElem)
MassUpper['NameList']=snname
MassMid=pd.DataFrame(np.zeros([11,35]),columns=['NameList']+ChosenElem)
MassMid['NameList']=snname
MassLower=pd.DataFrame(np.zeros([11,35]),columns=['NameList']+ChosenElem)
MassLower['NameList']=snname
for j in snname:
    for k in ChosenElem:
        MassData.loc[MassData['NameList']==j,k]=PredData.loc[PredData['NameList']==j,k]*ElemCal(Abundance=mdd116,Density=dens,date=19,elem=k)  
        MassUpper.loc[MassUpper['NameList']==j,k]=PredUpper.loc[PredUpper['NameList']==j,k]*ElemCal(Abundance=mdd116,Density=dens,date=19,elem=k)  
        MassMid.loc[MassMid['NameList']==j,k]=PredMid.loc[PredMid['NameList']==j,k]*ElemCal(Abundance=mdd116,Density=dens,date=19,elem=k)  
        MassLower.loc[MassLower['NameList']==j,k]=PredLower.loc[PredLower['NameList']==j,k]*ElemCal(Abundance=mdd116,Density=dens,date=19,elem=k)
MassData.to_csv('DataProduct/2019.3.3/MassData.csv')
MassLower.to_csv('DataProduct/2019.3.3/MassLower.csv')
MassMid.to_csv('DataProduct/2019.3.3/MassMid.csv')
MassUpper.to_csv('DataProduct/2019.3.3/MassUpper.csv')